# Importing libraries

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import pandas as pd
import time



# Extracting jobs links 

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Set the path to the chromedriver executable
PATH = r"C:\Program Files (x86)\chromedriver.exe"
s = Service(PATH)

# Initialize the Chrome WebDriver
driver = webdriver.Chrome(service=s)

# Open the webpage
driver.get("https://www.linkedin.com/jobs/search/?currentJobId=3959479897&distance=25&geoId=105015875&keywords=full%20remote&origin=JOBS_HOME_KEYWORD_HISTORY&refresh=true")

# Wait for the page to load
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'infinite-scroller__show-more-button')))

clicks = 0
max_clicks = 10  
initial_scrolls = 6  # Scroll down 6 times before trying to click the button
scroll_repeat_after_button = 1  # Scroll down 1 time after clicking the "Load More" button

# Initial scrolling to load content before interacting with the "Load More" button
for _ in range(initial_scrolls):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    WebDriverWait(driver, 10).until(lambda d: d.execute_script("return document.readyState") == "complete")
    time.sleep(2)  # Allow time for the scroll to complete and any new content to load

while clicks < max_clicks:
    try:
        # Wait for and click the "Load More" button
        load_more = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "infinite-scroller__show-more-button")))
        driver.execute_script("arguments[0].scrollIntoView(true);", load_more)  # Scroll to the "Load More" button
        load_more.click()
        clicks += 1
        print(f"Clicked 'Load More' {clicks} times.")
        
        # Scroll down the page once after clicking the button
        for _ in range(scroll_repeat_after_button):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            WebDriverWait(driver, 10).until(lambda d: d.execute_script("return document.readyState") == "complete")
            time.sleep(2)  # Allow time for the scroll to complete and any new content to load
    
    except (NoSuchElementException, TimeoutException) as e:
        # If the "Load More" button is not found or not interactable, stop scrolling
        print(f"No more 'Load More' button found or exception occurred: {e}")
        break

# After finishing the scrolling and clicking process, scrape the links
try:
    container = driver.find_element(By.CLASS_NAME, 'jobs-search__results-list')
    links = container.find_elements(By.TAG_NAME, "li")
    liste = [element.find_element(By.TAG_NAME, "a").get_attribute("href") for i, element in enumerate(links) if i <= 200]
    df = pd.DataFrame({'Links': liste})
    df.to_excel("links_sheet.xlsx", index=False)
    print("Links have been scraped and saved to 'links_sheet.xlsx'")
except Exception as e:
    print(f'Error in getting links: {e}')

# Close the driver
driver.quit()


<selenium.webdriver.remote.webelement.WebElement (session="6aa596ded0af2a3c6be1c24cbb67b146", element="f.B1F86FF3FAC5006DF4C8407FE359D2A3.d.6F74C0C04ACF4E5E916DE31661E7443A.e.92")>


PermissionError: [Errno 13] Permission denied: 'links_sheet.xlsx'

# Ectracting jobs informations

In [8]:
def clean_link(link):
    """
    Remove the subdomain prefix (like 'fr.', 'uk.') from the link if it is not 'www.'.

    Args:
    - link (str): The URL to be cleaned.

    Returns:
    - str: The cleaned URL.
    """
    # Check if 'www.' is in the link after '://'
    if '://www.' in link:
        return link
    
    # Find the position of '://'
    pos = link.find('://') + 3  # Position right after '://'
    
    # Split the URL into components
    base_part = link[pos:]
    
    # Check if the base part has a subdomain (like 'fr.' or 'uk.')
    if '.' in base_part and not base_part.startswith('www.'):
        # Find the first dot to determine where the subdomain ends
        first_dot = base_part.find('.')
        # Remove the subdomain prefix
        cleaned_url = link[:pos] + base_part[first_dot + 1:]
        return cleaned_url
    
    return link

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import pandas as pd

PATH = r"C:\Program Files (x86)\chromedriver.exe"
s = Service(PATH)

# Initialize the Chrome WebDriver
driver = webdriver.Chrome(service=s)
job_links = []
links = pd.read_excel("links_sheet.xlsx")
print("test1")
links_list = links['Links'].tolist()
print("test2")
job_title = []
company_name = []
company_link = []
description = []
Niveau_hierarchique = []
Type_emploi = []
Fonction = []
Secteurs = []

for i, link in enumerate(links_list):
    if i > 20 : 
        break
    try:
        driver.get(link)
        time.sleep(5)
        print("job___________ ", i)
        container = driver.find_element(By.CLASS_NAME, "core-rail")

        TOP_card = container.find_element(By.CLASS_NAME, "top-card-layout")
        job_title.append(TOP_card.find_element(By.CLASS_NAME, "top-card-layout__title").text)
        print(job_title)
        company_name.append(TOP_card.find_element(By.CLASS_NAME, "topcard__org-name-link").text)
        print(company_name)
        company_link.append(clean_link(TOP_card.find_element(By.CLASS_NAME, "topcard__org-name-link").get_attribute("href")))
        print(company_link)
        
        details_container = container.find_element(By.CLASS_NAME, "core-section-container__content")
        description.append(details_container.find_element(By.CLASS_NAME, "show-more-less-html__markup").text)
        print("description done")

        job_criterias_container = details_container.find_element(By.CLASS_NAME, "description__job-criteria-list")
        job_criteria_list = job_criterias_container.find_elements(By.CLASS_NAME, "description__job-criteria-item")
        
        # Initialize criteria with None
        criteria_values = {
            'Niveau_hierarchique': None,
            'Type_emploi': None,
            'Fonction': None,
            'Secteurs': None
        }

        for k, criteria in enumerate(job_criteria_list):
            if k == 0:
                criteria_values['Niveau_hierarchique'] = criteria.find_element(By.CLASS_NAME, 'description__job-criteria-text').text
            elif k == 1:
                criteria_values['Type_emploi'] = criteria.find_element(By.CLASS_NAME, 'description__job-criteria-text').text
            elif k == 2:
                criteria_values['Fonction'] = criteria.find_element(By.CLASS_NAME, 'description__job-criteria-text').text
            elif k == 3:
                criteria_values['Secteurs'] = criteria.find_element(By.CLASS_NAME, 'description__job-criteria-text').text
        
        Niveau_hierarchique.append(criteria_values['Niveau_hierarchique'])
        Type_emploi.append(criteria_values['Type_emploi'])
        Fonction.append(criteria_values['Fonction'])
        Secteurs.append(criteria_values['Secteurs'])
                
    except Exception as e:
        print(f"Error processing job {i}: {e}")
        job_title.append(None)
        company_name.append(None)
        company_link.append(None)
        description.append(None)
        Niveau_hierarchique.append(None)
        Type_emploi.append(None)
        Fonction.append(None)
        Secteurs.append(None)

df = pd.DataFrame({
    'Job title': job_title, 
    'company name': company_name, 
    'company link': company_link, 
    'Job description': description, 
    'Niveau hierarchique': Niveau_hierarchique, 
    'Type emploi': Type_emploi, 
    'Fonction': Fonction, 
    'Secteurs': Secteurs
})

df.to_excel("data_sheet.xlsx", index=False)
driver.quit()


test1
test2
job___________  0
['HR Recruitment Coordinator (Remote)']
['Kreativstorm']
['https://linkedin.com/company/kreativstorm?trk=public_jobs_topcard-org-name']
description done
job___________  1
['HR Recruitment Coordinator (Remote)', 'HR Consultant (Remote)']
['Kreativstorm', 'Kreativstorm']
['https://linkedin.com/company/kreativstorm?trk=public_jobs_topcard-org-name', 'https://linkedin.com/company/kreativstorm?trk=public_jobs_topcard-org-name']
description done
job___________  2
['HR Recruitment Coordinator (Remote)', 'HR Consultant (Remote)', 'Customer Support Representative']
['Kreativstorm', 'Kreativstorm', 'APOLOP2P']
['https://linkedin.com/company/kreativstorm?trk=public_jobs_topcard-org-name', 'https://linkedin.com/company/kreativstorm?trk=public_jobs_topcard-org-name', 'https://linkedin.com/company/apolop2p?trk=public_jobs_topcard-org-name']
description done
job___________  3
['HR Recruitment Coordinator (Remote)', 'HR Consultant (Remote)', 'Customer Support Representati

# Extracting companies informations 

In [11]:


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
import re

# Chemin vers ChromeDriver
PATH = r"C:\Program Files (x86)\chromedriver.exe"
s = Service(PATH)

# Charger les données depuis un fichier Excel
data = pd.read_excel("data_sheet.xlsx")
links = data['company link']

# Initialiser les listes pour stocker les données extraites
description = []
site_web = []
secteur = []
taille_entreprise = []
siege_social = []
type_entreprise = []
domaines = []
annee_fondation = []

def normalize_text(text):
    # Supprimer les caractères non alphanumériques et normaliser les espaces
    return re.sub(r'\W+', '', text).strip().lower()

for i,link in enumerate(links):


    try:
        # Initialiser le WebDriver pour chaque lien
        driver = webdriver.Chrome(service=s)
        driver.get(link)
        WebDriverWait(driver, 12).until(EC.presence_of_element_located((By.TAG_NAME, "body")))  # Attendre que la page soit complètement chargée

        # Gérer le bouton de fermeture de la fenêtre modale si présent
        try:
            button = WebDriverWait(driver, 8).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "modal__dismiss"))
            )
            button.click()
        except Exception as e:
            print(f"Aucun bouton de fermeture de modal trouvé: {e}")

        # Extraire la description
        try:
            description_element = WebDriverWait(driver, 8).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, '[data-test-id="about-us__description"]'))
            )
            description.append(description_element.text)
        except Exception as e:
            print(f"Erreur lors de l'extraction de la description: {e}")
            description.append("null")

        # Extraire plus de détails
        try:
            more_details_container = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.CLASS_NAME, "mt-6"))
            )

            dt_container = more_details_container.find_elements(By.TAG_NAME, 'dt')
            dd_container = more_details_container.find_elements(By.TAG_NAME, 'dd')

            # Dictionnaires pour stocker les champs et leurs listes correspondantes
            fields = {
                normalize_text("Site web"): site_web,
                normalize_text("Secteur"): secteur,
                normalize_text("Taille de l'entreprise"): taille_entreprise,
                normalize_text("Siège social"): siege_social,
                normalize_text("Type"): type_entreprise,
                normalize_text("Domaines"): domaines,
                normalize_text("Fondée en"): annee_fondation
            }

            # Extraire les données basées sur les paires dt/dd
            for dt, dd in zip(dt_container, dd_container):
                key = normalize_text(dt.text)
                value = dd.text.strip()
                if key in fields:
                    fields[key].append(value)
                else:
                    print(f"Champ non reconnu: {dt.text.strip()}")

            # S'assurer que tous les champs ont des valeurs (définir à "null" si manquant)
            for field in fields:
                if field not in [normalize_text(dt.text) for dt in dt_container]:
                    fields[field].append("null")

        except Exception as e:
            print(f"Erreur lors de l'extraction des détails: {e}")
            # Ajouter "null" pour chaque champ en cas d'erreur
            site_web.append("null")
            secteur.append("null")
            taille_entreprise.append("null")
            siege_social.append("null")
            type_entreprise.append("null")
            domaines.append("null")
            annee_fondation.append("null")

        # Fermer la fenêtre actuelle
        driver.quit()

    except Exception as e:
        print(f"Erreur lors de l'extraction des données pour le lien {link}: {e}")
        # Ajouter "null" pour chaque champ en cas d'erreur
        description.append("null")
        site_web.append("null")
        secteur.append("null")
        taille_entreprise.append("null")
        siege_social.append("null")
        type_entreprise.append("null")
        domaines.append("null")
        annee_fondation.append("null")

# Vérifier les longueurs des listes
lengths = [len(description), len(site_web), len(secteur), len(taille_entreprise), len(siege_social), len(type_entreprise), len(domaines), len(annee_fondation)]
max_length = max(lengths)

# Ajouter des valeurs "null" pour égaliser les longueurs des listes
for field in [description, site_web, secteur, taille_entreprise, siege_social, type_entreprise, domaines, annee_fondation]:
    while len(field) < max_length:
        field.append("null")

# Convertir les listes en DataFrame
scraped_data = pd.DataFrame({
    "Description de l'entreprise": description,
    'Site web': site_web,
    'Secteur': secteur,
    "Taille de l'entreprise": taille_entreprise,
    'Siège social': siege_social,
    'Type': type_entreprise,
    'Domaines': domaines,
    "Année de fondation": annee_fondation
})

# Charger les données existantes du fichier Excel
existing_data = pd.read_excel("data_sheet.xlsx")

# Ajouter de nouvelles colonnes aux données existantes
updated_data = pd.concat([existing_data, scraped_data], axis=1)

# Enregistrer les données mises à jour dans le fichier Excel existant
updated_data.to_excel("datasheet.xlsx", index=False)

print("Extraction et mise à jour des données terminées.")

# Quitter le WebDriver
driver.quit()


Erreur lors de l'extraction des données pour le lien nan: Message: invalid argument: missing command parameters
Stacktrace:
	GetHandleVerifier [0x00007FF70B7EEEA2+31554]
	(No symbol) [0x00007FF70B767ED9]
	(No symbol) [0x00007FF70B62872A]
	(No symbol) [0x00007FF70B696C9A]
	(No symbol) [0x00007FF70B6968D1]
	(No symbol) [0x00007FF70B5F2B75]
	GetHandleVerifier [0x00007FF70BAF8B1D+3217341]
	GetHandleVerifier [0x00007FF70BB45AE3+3532675]
	GetHandleVerifier [0x00007FF70BB3B0E0+3489152]
	GetHandleVerifier [0x00007FF70B89E776+750614]
	(No symbol) [0x00007FF70B77375F]
	(No symbol) [0x00007FF70B5F1B97]
	GetHandleVerifier [0x00007FF70BBA4348+3919848]
	BaseThreadInitThunk [0x00007FFB1AB47344+20]
	RtlUserThreadStart [0x00007FFB1C6A26B1+33]

Extraction et mise à jour des données terminées.
